In [16]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
import numpy as np
import sys

In [ ]:
# Función para cargar y procesar una hoja
def cargar_hoja(archivo, hoja, skiprows, moneda):
    df = pd.read_excel(archivo, sheet_name=hoja, skiprows=skiprows, usecols=lambda x: 'Unnamed' not in x)
    df["Moneda"] = moneda
    return df.fillna(method='ffill')

# Cargar todas las hojas
df_eeff_mn = cargar_hoja("Boletín Bancos MN.xlsm", "1. EEFF", 23, "MN")
df_eeff_me = cargar_hoja("Boletín Bancos ME.xlsm", "1. EEFF", 23, "ME")

df_cartera_mn = cargar_hoja("Boletín Bancos MN.xlsm", "3. Cartera", 23, "MN")
df_cartera_me = cargar_hoja("Boletín Bancos ME.xlsm", "3. Cartera", 23, "ME")

df_sector_mn = cargar_hoja("Boletín Bancos MN.xlsm", "5. Cred. por sector", 21, "MN")
df_sector_me = cargar_hoja("Boletín Bancos ME.xlsm", "5. Cred. por sector", 21, "ME")

df_actividad_mn = cargar_hoja("Boletín Bancos MN.xlsm", "6. Cred. por acti", 17, "MN")
df_actividad_me = cargar_hoja("Boletín Bancos ME.xlsm", "6. Cred. por acti", 17, "ME")

# Unir MN y ME por tipo
df_eeff = pd.concat([df_eeff_mn, df_eeff_me], ignore_index=True)
df_cartera = pd.concat([df_cartera_mn, df_cartera_me], ignore_index=True)
df_sector = pd.concat([df_sector_mn, df_sector_me], ignore_index=True)
df_actividad = pd.concat([df_actividad_mn, df_actividad_me], ignore_index=True)

# Guardar en un nuevo archivo Excel con varias hojas
with pd.ExcelWriter("Boletin_Procesado.xlsx", engine='openpyxl') as writer:
    df_eeff.to_excel(writer, sheet_name="EEFF", index=False)
    df_cartera.to_excel(writer, sheet_name="Cartera", index=False)
    df_sector.to_excel(writer, sheet_name="Cred_por_sector", index=False)
    df_actividad.to_excel(writer, sheet_name="Cred_por_actividad", index=False)

In [17]:
# Cargar todas las hojas del archivo
excel_file = "Boletin_Procesado.xlsx"
sheets_dict = pd.read_excel(excel_file, sheet_name=None)  # Lee todas las hojas como un diccionario

# Guardar cada hoja en un DataFrame separado
df_eeff = sheets_dict["EEFF"]
df_cartera = sheets_dict["Cartera"]
df_sector = sheets_dict["Cred_por_sector"]
df_actividad = sheets_dict["Cred_por_actividad"]

# Mostrar para confirmar
print("Hojas cargadas:")
for nombre in sheets_dict:
    print(f"- {nombre}")

Hojas cargadas:
- EEFF
- Cartera
- Cred_por_sector
- Cred_por_actividad


In [20]:
# Filtrar solo los datos del Sector Agri
df_sector_agri = df_sector[df_sector["Sector E."].str.contains("AGRICULTURA", na=False)]

# Cambiar el formato fecha
df_sector_agri['Fecha'] = pd.to_datetime(df_sector_agri['Fecha'].astype(str) + '/01', format='%Y/%m/%d')

C:\Users\Mathias\AppData\Local\Temp\ipykernel_3796\157564836.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sector_agri['Fecha'] = pd.to_datetime(df_sector_agri['Fecha'].astype(str) + '/01', format='%Y/%m/%d')


In [21]:
df_sector_agri.head()

,Fecha,Sector E.,Atlas,Bancop,Basa,BNA,BNF,Citibank,Continental,Do Brasil,...,Interfisa,Itaú,Regional,Río,Solar,Sudameris,UENO,Visión,Zeta,Moneda
2,2016-01-01,AGRICULTURA,41185.409522,16045.479312,17860.375384,5423.163206,605493.388886,0.000000,206516.958567,26160.829795,...,126020.851705,355901.509195,260824.451256,67891.391814,0.0,14842.417765,0.0,143240.695952,0.0,MN
16,2016-02-01,AGRICULTURA,40320.707056,16072.839808,16897.634363,5224.573884,595339.340213,49.157955,199867.699827,25966.528767,...,126827.897305,356697.465315,259562.275778,78532.289301,0.0,15481.779517,0.0,147510.438844,0.0,MN
30,2016-03-01,AGRICULTURA,36088.521845,19621.826036,15734.211851,4947.564455,550232.396346,0.000000,201133.264315,25856.218151,...,128616.005940,352712.517888,254525.302471,78261.183487,0.0,17331.017130,0.0,147654.306796,0.0,MN
44,2016-04-01,AGRICULTURA,36598.922049,19421.659220,17480.058018,4539.595701,532903.399800,20.676026,205748.990976,25990.324315,...,133895.362096,316883.455978,248998.458379,78475.409733,0.0,18618.970931,0.0,148863.488089,0.0,MN
58,2016-05-01,AGRICULTURA,31842.404714,25582.789163,16980.924473,5093.816257,521225.886836,0.000000,194543.983377,25975.473115,...,132491.779313,285150.153921,259406.557463,78348.309923,0.0,20393.696440,0.0,149730.149596,0.0,MN


In [ ]:
graficos de pili
graficos de pili